In [7]:
import pandas as pd
import numpy as np

# 1. 분리된 테스트 데이터 로드
#test_data = pd.read_csv('../ml_without_phone/splited_data/gym_test.csv')
test_data = pd.read_csv('../ml_without_phone/splited_data/test_msh.csv')

print("=== 원본 데이터 확인 ===")
print(f"데이터 크기: {test_data.shape}")
print(f"컬럼: {list(test_data.columns)}")
print("\n첫 5개 행:")
print(test_data.head())

# 2. 라벨 디코딩 (원본 값으로 복원)
korean_data = test_data.copy()

# Phone 컬럼이 있으면 제거 (ml_without_phone 프로젝트)
if 'Phone' in korean_data.columns:
    korean_data = korean_data.drop('Phone', axis=1)
    print("\n📱 Phone 컬럼을 제거했습니다.")

# 성별 복원
korean_data['성별'] = korean_data['gender'].map({0: '여성', 1: '남성'})

# 헬스장 근처 거주 복원
korean_data['헬스장 근처 거주'] = korean_data['Near_Location'].map({0: '아니오', 1: '예'})

# 파트너 프로그램 복원
korean_data['파트너 프로그램'] = korean_data['Partner'].map({0: '아니오', 1: '예'})

# 친구 할인 복원
korean_data['친구 할인'] = korean_data['Promo_friends'].map({0: '아니오', 1: '예'})

# 그룹 수업 참여 복원
korean_data['그룹 수업 참여'] = korean_data['Group_visits'].map({0: '아니오', 1: '예'})

# 계약 기간 복원
korean_data['계약 기간'] = korean_data['Contract_period'].map({
    1: '1개월', 6: '6개월', 12: '12개월'
})

# 이탈 여부 복원
korean_data['이탈 여부'] = korean_data['Churn'].map({0: '유지', 1: '이탈'})

# 숫자 컬럼들 이름 변경 및 포맷팅
korean_data['나이'] = korean_data['Age'].round(0).astype(int)
korean_data['평균 추가 요금'] = korean_data['Avg_additional_charges_total'].round(2)
korean_data['계약 만료까지 개월'] = korean_data['Month_to_end_contract'].round(1)
korean_data['이용 기간 개월'] = korean_data['Lifetime'].round(0).astype(int)
korean_data['전체 수업 참여 빈도'] = korean_data['Avg_class_frequency_total'].round(3)
korean_data['이번달 수업 참여 빈도'] = korean_data['Avg_class_frequency_current_month'].round(3)

# 3. 원본 영어 컬럼 제거하고 한글 컬럼만 유지
korean_columns = [
    '성별', '헬스장 근처 거주', '파트너 프로그램', '친구 할인',
    '그룹 수업 참여', '계약 기간', '나이', '평균 추가 요금',
    '계약 만료까지 개월', '이용 기간 개월', '전체 수업 참여 빈도',
    '이번달 수업 참여 빈도', '이탈 여부'
]

# 예측을 위한 원본 숫자 데이터도 별도로 유지 (모델 입력용)
# 중요: 모델 학습 시와 동일한 컬럼 순서를 유지해야 함
model_input_columns = [
    'gender', 'Near_Location', 'Partner', 'Promo_friends',
    'Contract_period', 'Group_visits', 'Age', 'Avg_additional_charges_total',
    'Month_to_end_contract', 'Lifetime', 'Avg_class_frequency_total',
    'Avg_class_frequency_current_month'  # Churn 제외 (타겟 변수)
]

# Phone 컬럼 제거된 데이터에서 X, y 분리
X_columns = [col for col in model_input_columns if col in test_data.columns]
y_column = 'Churn'

print(f"\n=== 모델 입력 컬럼 확인 ===")
print(f"사용할 feature 컬럼: {X_columns}")
print(f"컬럼 개수: {len(X_columns)}")

# 4. 두 가지 버전 저장
# 4-1. 한글 표시용 데이터
korean_display_data = korean_data[korean_columns].copy()
korean_display_data.to_csv('../ml_without_phone/splited_data/gym_test_korean.csv',
                          index=False, encoding='utf-8-sig')

# 4-2. 모델 입력용 데이터 (X features + y target)
model_input_data = pd.DataFrame()
for col in X_columns:
    model_input_data[col] = test_data[col]
model_input_data[y_column] = test_data[y_column]

model_input_data.to_csv('../ml_without_phone/splited_data/gym_test_for_model.csv',
                       index=False)

print("\n=== 한글 데이터 변환 완료 ===")
print(f"한글 표시용 데이터: {korean_display_data.shape}")
print(f"모델 입력용 데이터: {model_input_data.shape}")

print("\n=== 모델 입력용 데이터 컬럼 순서 ===")
print(f"X features: {list(model_input_data.columns[:-1])}")
print(f"y target: {model_input_data.columns[-1]}")

print("\n=== 한글 데이터 미리보기 ===")
print(korean_display_data.head())

print("\n=== 파일 저장 완료 ===")
print("1. gym_test_korean.csv - 한글 표시용")
print("2. gym_test_for_model.csv - 모델 예측용 (정확한 컬럼 순서)")

# 5. 각 컬럼의 고유값 확인
print("\n=== 변환된 값들 확인 ===")
for col in ['성별', '헬스장 근처 거주', '계약 기간', '이탈 여부']:
    if col in korean_display_data.columns:
        print(f"{col}: {korean_display_data[col].unique()}")

=== 원본 데이터 확인 ===
데이터 크기: (800, 13)
컬럼: ['gender', 'Near_Location', 'Partner', 'Promo_friends', 'Contract_period', 'Group_visits', 'Age', 'Avg_additional_charges_total', 'Month_to_end_contract', 'Lifetime', 'Avg_class_frequency_total', 'Avg_class_frequency_current_month', 'Churn']

첫 5개 행:
   gender  Near_Location  Partner  Promo_friends  Contract_period  \
0       1              1        0              0               12   
1       1              1        0              0               12   
2       0              1        0              0                1   
3       1              1        0              0                1   
4       0              1        0              0                1   

   Group_visits  Age  Avg_additional_charges_total  Month_to_end_contract  \
0             0   29                     80.694518                   11.0   
1             0   32                     88.494562                   12.0   
2             1   32                    102.616374             